<img src="https://julialang.org/v2/img/logo.svg" width=250></img>

> scientific computing, machine learning, data mining, large-scale linear algebra, distributed and parallel computing 

> We want a language that’s **open source**, with a liberal license. 
> We want the **speed** of C with the **dynamism** of Ruby. 
> We want a language that’s **homoiconic, with true macros** like Lisp, 
> but with obvious, **familiar mathematical notation** like Matlab. 
> We want something as usable for general programming as Python, 
> as **easy for statistics** as R, as natural for **string processing** as Perl, 
> as powerful for **linear algebra** as Matlab, as good at **gluing programs together** as the shell. 
> Something that is **dirt simple** to learn, yet keeps the most serious hackers happy. 
> We want it **interactive** and we want it **compiled**.

## Julia - Main Features
- Dynamic programming language for technical computing
- Strongly typed with Any-type and type inference
- JIT compilation to machine code (using LLVM)
- Matlab-like notation/convenience for arrays
- Advanced features:
    - Multiple dispatch
    - Matrix operators for all LAPACK types
    - Sparse matrices and operators
    - Parallel processing
    - Meta programming
- Developed at MIT since 2012, MIT license

## MATLAB 
- no nice structure for developing code 
- lack of namespacing for packages
- ambiguity between indexing and function

## MATLAB and Julia ODE solver benchmark
![](https://raw.githubusercontent.com/JuliaDiffEq/MATLABDiffEq.jl/master/assets/matlab_bench.png)

---

In [ ]:
using ProgressMeter
using Statistics
using Printf
using DifferentialEquations
using Plots
gr(fmt="png", size=(400, 300))

In [ ]:
@doc raw"""**activationFunction**(Vhalf, k)

activation function of the ion channel, as boltzmann function.

$$m_\infty(V) = \frac{1}{1 + \exp(\frac{V_{\text{half}} - V}{k})}$$
"""
function activationFunction(Vhalf::Float64, k)
    return (V) -> 1 / (1+exp((Vhalf-V)/k))
end

function activationFunction(Vhalf::Int64, k)
    return (V) -> 1 / (1+exp((Vhalf-V)/k))
end

@doc raw"""**timeConstant**(Vmax, sigma, Camp, Cbase)

time constant of the ion channel, as gaussian function.

$$\tau(V) = C_{\text{base}} + C_{\text{amp}} \exp(\frac{- (V_{\text{max}} - V) ^ 2}{\sigma ^ 2})$$
"""
function timeConstant(Vmax, sigma, Camp, Cbase)
    return (V) -> Cbase + Camp * exp(-((Vmax - V)/sigma)^2)
end

---

In [ ]:
@doc raw"""
## $I_{\text{Na,p}}+I_{\text{K}}$-model

$$C\dot{V} = I - g_{\text{L}}(V - E_{\text{L}}) - g_{\text{Na}}m_\infty(V)(V-E_{\text{Na}}) - g_{\text{K}}n(V - E_{\text{K}})$$
$$\dot{n} = \frac{n_\infty(V) - n}{\tau_n(V)}$$

- u = [V, m, n, I]
"""
function persistent_sodium_plus_potassium_model(du, u, p, t)
    
    E_Na = 60.0
    E_K  = -90.0
    E_L  = -80.0
    
    m_infty = activationFunction(p.m_half, p.m_k)
    # tau_m = (V) -> 0.8
    
    n_infty = activationFunction(p.n_half, p.n_k)
    tau_n = (V) -> 1.0
    
    g_L = 8.0
    g_Na= p.g_Na #20.0 # 10
    g_K = p.g_K #10.0
    
    if t > 40
        u[4] = p.I
    else
        u[4] = 0
    end
    
    du[1] = u[4] - g_L * (u[1] - E_L) - g_Na * m_infty(u[1]) * (u[1] - E_Na) - g_K * u[3] * (u[1] - E_K)
    u[2] = m_infty(u[1])
    du[3] = (n_infty(u[1]) - u[3]) / tau_n(u[1])
end

In [ ]:
m_infty = activationFunction(_p.m_half, _p.m_k)
20 * m_infty(0) * (80)

In [ ]:
?persistent_sodium_plus_potassium_model

In [ ]:
u0 = [-66.0, 0.045, 0.0, 0.0]
tspan = (0.0, 100.0)

_p = (I=10.0, g_Na=20.0, g_K=15.0, 
    m_half=-20.0, m_k = 15.0, 
    n_half=-25.0, n_k=5.0)

prob = ODEProblem(persistent_sodium_plus_potassium_model, u0, tspan, _p)
sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8);
_trace = [item[1] for item in sol(tspan[1]:0.1:tspan[2]).u];

In [ ]:
a = plot(sol, vars=(1), size=(800, 300), title="$(_p)", legend=nothing, ylim=(-80, 20))
vline!([40])

plot(activationFunction(_p.m_half, _p.m_k), -100, 20, color=nordtheme.frost2, linewidth=3, xlabel="V")
b = plot!(activationFunction(_p.n_half, _p.n_k),-100, 20, color=nordtheme.aurora2, linewidth=3, legend=nothing)
c = bar(["g_Na", "g_K", "g_L"], [_p.g_Na, _p.g_K, 8], color=[nordtheme.frost2, nordtheme.aurora2, nordtheme.aurora1], legend=nothing, ylim=(0, 25))
d = plot(b, c)

# plot!(sol, vars=(3))

plot(a, d, layout=@layout([a;b]), size=(800, 600))
# savefig("export/preview.pdf")

In [ ]:
tmp = zeros(10000, 20000)
@time for i = 1:1000
    for j = 1:2000
        tmp[i, j] = rand()
    end
end

In [ ]:
tmp = zeros(10000, 20000)
@time Threads.@threads for i = 1:1000
    for j = 1:2000
        tmp[i, j] = rand()
    end
end

In [ ]:
function detect_cross_pnt(arr, thr, way=:up, gap=1)
    _idx_repo = []
    try
        idx = findall(_trace .> -20)
        idx_diff = findall(diff(idx) .> 1)
        _idx_repo = [idx[1]; idx[idx_diff]; idx[idx_diff .+ 1]; idx[end]]
    catch BoundError
        return nothing
    end
    
    if way == :up
        _check = (x, i) -> x[i-1] < thr < x[i] #< x[i+1]
    elseif way == :down
        _check = (x, i) -> x[i-1] > x[i] > x[i+1]
    else
        return nothing
    end
    
    sort!(_idx_repo)
    _result = Vector{Int}()
    _previous = -9999
    
    for idx in _idx_repo
            if _check(arr, idx) && (idx - _previous > gap)
                _result = [_result; idx]
                _previous = idx
            end
    end
    
    _result
end

_x = tspan[1]:0.1:tspan[2]
_xing = detect_cross_pnt(_trace, -20)
plot(_x, _trace, legend=nothing, size=(800, 400), linewidth=3)
hline!([-20])
vline!(_x[_xing], linewidth=3)
# savefig("export/spike_detection.pdf")

In [ ]:
plot(sol, vars=(1, 3), size=(300, 300), legend=nothing, xlabel="voltage", ylabel="n")

In [ ]:
p = Progress(101, 1)
u0 = [-66.0, 0.045, 0.0, 0.0]
tspan = (0.0, 100.0)
_trace = 0
_result = Array{Float64, 2}(undef, 101, 0)

for j = 1:101
    _fr = Vector{Float64}()
    for i = 1:101
        # I ~ 5, 15; 10
        # g_Na ~ 18, 22; 20
        # g_K ~ 8, 12; 10
        # m_half ~ -22, -18; -20
        # n_half ~ -27, -23; -25
        # m_k ~ 13, 17;15.0
        # n_k ~ 3, 7; 5.0

        _p = (I=(i-1)/5, g_Na=22, g_K=10, m_half=-20, m_k = (j-1)/50+14, n_half=-25.0, n_k=5.0)

        prob = ODEProblem(persistent_sodium_plus_potassium_model, u0, tspan, _p)
        sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8);
        _trace = [item[1] for item in sol(tspan[1]:0.1:tspan[2]).u];

        _x = tspan[1]:0.1:tspan[2]
        _xing = detect_cross_pnt(_trace, -20)
        
        if _xing == nothing
            _fr = [_fr; 0]
#             println("zeros")
        else
            _x_cross = _x[_xing]
            _x_cross = _x_cross[_x_cross .> 40]
            _fr = [_fr; 1e3 / mean(diff(_x_cross))]
        end
    end
    _result = [_result _fr]
    next!(p)
end

anim = @animate for _sweep_idx = 1:101
    _title = @sprintf "km = %.3f" (_sweep_idx-1)/40+18
    plot(0:0.2:20, _result[:, _sweep_idx], legend=nothing, ylim=(-1, 300),
        title=_title, xlabel="input current", ylabel="firing rate", 
        color=nordtheme.frost2, linewidth=3)
end
gif(anim, "export/FI_km.gif", fps=12);

plot(legend=nothing, color=:green, size=(900, 600), ylim=(-1, 300), xlabel="input current", ylabel="firing rate",)
for (idx, _ridx) = enumerate(1:10:size(_result, 2))
    plot!(0:0.2:20, _result[:, _ridx], alpha=(idx-1)/15+.6, color=nordtheme.frost2,linewidth=3)
end
plot!()
savefig("export/FI_km.pdf")

---

In [ ]:
p = Progress(101, 1)   
anim = @animate for i = 1:101
    # I ~ 5, 15; 10
    # g_Na ~ 18, 22; 20
    # g_K ~ 8, 12
    # m_half ~ -22, -18; -20
    # n_half ~ -27, -23; -25
    # m_k ~ 13, 17;15.0
    # n_k ~ 3, 7; 5.0
    
    _p = (I=(i-1)/10+5.0, g_Na=20.0, g_K=10, m_half=-20.0, m_k = 15, n_half=-25.0, n_k=5.0)

    prob = ODEProblem(persistent_sodium_plus_potassium_model, u0, tspan, _p)
    sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8);
    

    a = plot(sol, vars=(1), size=(800, 300), title="$(_p)", legend=nothing, ylim=(-80, 20))
    vline!([40])

    plot(activationFunction(_p.m_half, _p.m_k), -100, 20, color=nordtheme.frost2, linewidth=3, xlabel="V")
    b = plot!(activationFunction(_p.n_half, _p.n_k),-100, 20, color=nordtheme.aurora2, linewidth=3, legend=nothing)
    c = bar(["g_Na", "g_K", "g_L"], [_p.g_Na, _p.g_K, 8], color=[nordtheme.frost2, nordtheme.aurora2, nordtheme.aurora1], legend=nothing, ylim=(0, 25))
    d = plot(b, c)

    # plot!(sol, vars=(3))

    plot(a, d, layout=@layout([a;b]), size=(800, 600))
    next!(p)
end

gif(anim, "./export/n_k_3_7.gif", fps = 24);

---

In [ ]:
_p = (I=0.0, g_Na=20.0, g_K=15.0, m_half=-20.0, m_k = 15.0, n_half=-25.0, n_k=5.0)
E_Na = 60.0
E_K  = -90.0
E_L  = -80.0
g_L = 8.0

m_infty = activationFunction(_p.m_half, _p.m_k)

F(v, n) = _p.I - g_L * (v - E_L) - _p.g_Na * m_infty(v) * (v - E_Na) - _p.g_K * n * (v - E_K)
G(v, n) = activationFunction(_p.n_half, _p.n_k)(v) - n

_v=collect(-80:4:20)
_n=collect(0:0.05:1)
mesh = [repeat(_v, 1, size(_n, 1))[:] repeat(_n, 1, size(_v, 1))'[:]]
quiver_x = zeros(size(mesh, 1))
quiver_y = zeros(size(mesh, 1))

for i = 1:size(mesh, 1)
    quiver_x[i] = F(mesh[i,1], mesh[i,2])
    quiver_y[i] = G(mesh[i,1], mesh[i,2])
end

_e = sqrt.(quiver_y .^ 2 .+ quiver_x.^ 2)
# quiver_x = quiver_x ./ _e
# quiver_y = quiver_y ./ _e;

quiver(mesh[:, 1], mesh[:, 2], quiver=(quiver_x/200, quiver_y/200),
    xlim=(-80, 20), ylim=(0, 1), size=(600,600))